In [6]:
import numpy as np
from keras_transformer import get_model, decode

In [18]:
with open('../Dataset/English_Hindi.txt', mode = 'r') as f:
    data = f.readlines()

source_tokens = [i.split("	")[0].split(' ') for i in data[0:10]]
target_tokens = [i.split("	")[1].split(' ') for i in data[0:10]]

In [19]:
# Generate dictionaries
def build_token_dict(token_list):
    token_dict = {
        '<PAD>': 0,
        '<START>': 1,
        '<END>': 2,
    }
    for tokens in token_list:
        for token in tokens:
            if token not in token_dict:
                token_dict[token] = len(token_dict)
    return token_dict

source_token_dict = build_token_dict(source_tokens)
target_token_dict = build_token_dict(target_tokens)
target_token_dict_inv = {v: k for k, v in target_token_dict.items()}

# Add special tokens
encode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in source_tokens]
decode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in target_tokens]
output_tokens = [tokens + ['<END>', '<PAD>'] for tokens in target_tokens]

# Padding
source_max_len = max(map(len, encode_tokens))
target_max_len = max(map(len, decode_tokens))

encode_tokens = [tokens + ['<PAD>'] * (source_max_len - len(tokens)) for tokens in encode_tokens]
decode_tokens = [tokens + ['<PAD>'] * (target_max_len - len(tokens)) for tokens in decode_tokens]
output_tokens = [tokens + ['<PAD>'] * (target_max_len - len(tokens)) for tokens in output_tokens]

encode_input = [list(map(lambda x: source_token_dict[x], tokens)) for tokens in encode_tokens]
decode_input = [list(map(lambda x: target_token_dict[x], tokens)) for tokens in decode_tokens]
decode_output = [list(map(lambda x: [target_token_dict[x]], tokens)) for tokens in output_tokens]

# Build & fit model
model = get_model(
    token_num=max(len(source_token_dict), len(target_token_dict)),
    embed_dim=32,
    encoder_num=2,
    decoder_num=2,
    head_num=4,
    hidden_dim=128,
    dropout_rate=0.05,
    use_same_embed=False,  # Use different embeddings for different languages
)
model.compile('adam', 'sparse_categorical_crossentropy')
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Encoder-Input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 Encoder-Token-Embedding (Embed  [(None, None, 32),  480         ['Encoder-Input[0][0]']          
 dingRet)                        (15, 32)]                                                        
                                                                                                  
 Encoder-Embedding (TrigPosEmbe  (None, None, 32)    0           ['Encoder-Token-Embedding[0][0]']
 dding)                                                                                           
                                                                                            

In [20]:
%%time
model.fit(
    x=[np.array(encode_input * 1024), np.array(decode_input * 1024)],
    y=np.array(decode_output * 1024),
    epochs=10,
    batch_size=32,
)

Epoch 1/10
320/320 [==============================] - 16s 17ms/step - loss: 0.5669
Epoch 2/10
320/320 [==============================] - 5s 16ms/step - loss: 0.1369
Epoch 3/10
320/320 [==============================] - 5s 16ms/step - loss: 0.1262
Epoch 4/10
320/320 [==============================] - 5s 16ms/step - loss: 0.1244
Epoch 5/10
320/320 [==============================] - 5s 16ms/step - loss: 0.1236
Epoch 6/10
320/320 [==============================] - 5s 16ms/step - loss: 0.1232
Epoch 7/10
320/320 [==============================] - 5s 16ms/step - loss: 0.1226
Epoch 8/10
320/320 [==============================] - 5s 16ms/step - loss: 0.1226
Epoch 9/10
320/320 [==============================] - 5s 16ms/step - loss: 0.1224
Epoch 10/10
320/320 [==============================] - 5s 16ms/step - loss: 0.1224
CPU times: user 1min 55s, sys: 8.97 s, total: 2min 4s
Wall time: 1min 2s


In [23]:
# Predict
decoded = decode(
    model,
    encode_input,
    start_token=target_token_dict['<START>'],
    end_token=target_token_dict['<END>'],
    pad_token=target_token_dict['<PAD>'],
)
for i in decoded:
    print(' '.join(map(lambda x: target_token_dict_inv[x], i[1:-1])))

वाह!

बचाओ!

छलांग.

छलांग.

छलांग.

नमस्कार।

नमस्कार।

चियर्स!

चियर्स!

समझे कि नहीं?

